In [1]:
#붓꽃의 종(Species)을 분류해보자
#데이터의 결측치, 이상치에 대해 처리하고
#분류모델을 사용하여 정확도, F1 score, AUC 값을 산출하시오.
#제출은 아래 예시와 같이 result변수를 가진 dataframe 형태로 제출하시오

In [2]:
# 답안 제출 참고
# 아래 코드는 예시이며 변수명 등 개인별로 변경하여 활용
# write.csv(data.frame변수,"result.csv",row.names = FALSE)

In [3]:
 ###############  실기환경 복사 영역  ###############
 import pandas as pd
 import numpy as np
 # 실기 시험 데이터셋으로 셋팅하기 (수정금지)
 from sklearn.datasets import load_iris
 # Iris 데이터셋을 로드
 iris = load_iris()
 x = pd.DataFrame(iris.data, columns=['sepal_length', 'sepal_width', 'petal_length','petal_width'])
 y = iris.target   # 'setosa'=0, 'versicolor'=1, 'virginica'=2
 y = np.where(y>0, 1, 0) # setosa 종은 0, 나머지 종은 1로 변경
# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
 from sklearn.model_selection import train_test_split
 x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    stratify=y,
                                                    random_state=2023)
 x_test = pd.DataFrame(x_test)
 x_train = pd.DataFrame(x_train)
 y_train = pd.DataFrame(y_train)
 y_train.columns = ['species']
 x_test['sepal_length'].iloc[0] = None
 x_train['sepal_length'].iloc[0] = None
 
 # 이상치 삽입
 x_train['sepal_width'].iloc[0] = 150
 ###############  실기환경 복사 영역  ###############

C:\Users\user\AppData\Local\Temp\ipykernel_12396\1209920749.py:20: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_test['sepal_length'].iloc[0] = None
C:\Users\user\AppData\Local\Temp\ipykernel_12396\1209920749.py:21: FutureWarning: ChainedA

In [4]:
x_train.head()


,sepal_length,sepal_width,petal_length,petal_width
2,NaN,150.0,1.3,0.2
49,5.0,3.3,1.4,0.2
66,5.6,3.0,4.5,1.5
118,7.7,2.6,6.9,2.3
138,6.0,3.0,4.8,1.8


In [17]:
y_train.head()

,species
98,1
51,1
25,1
15,1
102,1


In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(120, 4)
(120, 1)
(30, 4)


In [6]:
print(x_train.isnull().sum())
print(y_train.isnull().sum())
print(x_test.isnull().sum())

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64
sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64


In [7]:
print(x_train.describe())
print(y_train.describe())
print(x_test.describe())

       sepal_length  sepal_width  petal_length  petal_width
count    119.000000     120.0000    120.000000   120.000000
mean       5.920168       4.2950      3.816667     1.226667
std        0.841667      13.4191      1.798848     0.780512
min        4.300000       2.2000      1.100000     0.100000
25%        5.150000       2.8000      1.575000     0.300000
50%        6.000000       3.0000      4.400000     1.350000
75%        6.500000       3.4000      5.225000     1.800000
max        7.900000     150.0000      6.900000     2.500000
          species
count  120.000000
mean     0.666667
std      0.473381
min      0.000000
25%      0.000000
50%      1.000000
75%      1.000000
max      1.000000
       sepal_length  sepal_width  petal_length  petal_width
count     29.000000    30.000000     30.000000     30.00000
mean       5.596552     3.000000      3.523333      1.09000
std        0.709367     0.522593      1.631518      0.68549
min        4.600000     2.000000      1.000000      0.1000

In [8]:
# 결측치 처리

median1 = x_train['sepal_length'].median()
median2 = x_test['sepal_length'].median()

x_train = x_train.fillna(median1)
x_test = x_test.fillna(median2)

In [9]:
print(x_train.isnull().sum())
print(y_train.isnull().sum())
print(x_test.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64


In [10]:
q1 = x_train['sepal_width'].quantile(.25)
q3 = x_train['sepal_width'].quantile(.75)

iqr = q3 - q1

upper = q3 + (iqr*1.5)
lower = q1 - (iqr*1.5)

cond1 = x_train['sepal_width'] > upper
cond2 = x_train['sepal_width'] < lower

outlier = x_train[(cond1 | cond2)]
print(outlier)


    sepal_length  sepal_width  petal_length  petal_width
2            6.0        150.0           1.3          0.2
15           5.7          4.4           1.5          0.4


In [11]:
x_train['sepal_width'].sort_values(ascending=False)


2     150.0
15      4.4
14      4.0
5       3.9
16      3.9
      ...  
53      2.3
87      2.3
41      2.3
62      2.2
68      2.2
Name: sepal_width, Length: 120, dtype: float64

In [12]:
x_train['sepal_width'] = x_train['sepal_width'].replace([150.0, 4.4],4.0)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y['species'], random_state = 2024)


In [16]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(96, 4)
(24, 4)
(96, 1)
(24, 1)


In [19]:
# 랜덤포레스트(분류)
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,max_depth=10 ,random_state=2024)
rfc.fit(x_train, y_train)
pred = rfc.predict(x_val)

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
matrix = confusion_matrix(y_val, pred)
acc = accuracy_score(y_val, pred)
f1 = f1_score(y_val, pred)
roc_auc = roc_auc_score(y_val, pred)

print(matrix, acc, f1, roc_auc)



[[ 9  0]
 [ 0 15]] 1.0 1.0 1.0


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [22]:
from sklearn.svm import SVC

svc = SVC(kernel = 'linear')
svc.fit(x_train,y_train)
pred2 = svc.predict(x_val)

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
matrix_2 = confusion_matrix(y_val, pred)
acc_2 = accuracy_score(y_val, pred)
f1_2 = f1_score(y_val, pred)
roc_auc_2 = roc_auc_score(y_val, pred)

print(matrix_2, acc_2, f1_2, roc_auc_2)


[[ 9  0]
 [ 0 15]] 1.0 1.0 1.0


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
pred3 = rfc.predict(x_test)

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
matrix_3 = confusion_matrix(y_test, y_test)
acc_3 = accuracy_score(y_test, y_test)
f1_3 = f1_score(y_test, y_test)
roc_auc_3 = roc_auc_score(y_test, y_test)

print(matrix_3, acc_3, f1_3, roc_auc_3)

[[10  0]
 [ 0 20]] 1.0 1.0 1.0


In [ ]:
answer = pd.DataFrame({'pred': pred3})
print(answer)


    pred
0      1
1      1
2      1
3      0
4      1
5      1
6      1
7      0
8      1
9      0
10     1
11     0
12     1
13     1
14     0
15     1
16     1
17     0
18     1
19     1
20     1
21     0
22     0
23     1
24     1
25     0
26     1
27     0
28     1
29     1


In [33]:
answer.to_csv('result.csv', index = False)
print(answer)

    pred
0      1
1      1
2      1
3      0
4      1
5      1
6      1
7      0
8      1
9      0
10     1
11     0
12     1
13     1
14     0
15     1
16     1
17     0
18     1
19     1
20     1
21     0
22     0
23     1
24     1
25     0
26     1
27     0
28     1
29     1


In [ ]:
# 답안 제출 참고
# 아래 코드는 예시이며 변수명 등 개인별로 변경하여 활용
# write.csv(data.frame변수,"result.csv",row.names = FALSE)